# Endemo 2 - Useful Energy Demand Model Rework

## Industry Sector



### Import Packages

In [12]:
import pathlib

import xlrd as x
import pandas as pd
from pathlib import Path
import os

### Import Custom Classes

In [13]:
from control_parameters import ControlParameters
from industry_sector import Industry
from household_sector import Household
from transport_sector import Transport
from commercial_trade_services_sector import CommercialTradeServices
from countries import Country
from product import Product

### Set Input Paths

In [14]:
# change path to point to your endemo2 folder

super_path = Path(os.path.abspath(''))
input_path = super_path / 'input'
print("Input folder at " + str(input_path))

general_path = input_path / 'general'
industry_path = input_path / 'industry'


Input folder at C:\Users\Chris\Documents\Master_Semester_1\IDP\endemo2\input


### Associate strings with code

SECTOR: necessary to connect the activated sectors in Set_and_Control_Parameters.xlsl with the code execution

In [15]:
SECTOR = dict({"industry":Industry(input_path / "industry"), "households":Household(), "transport":Transport(), "commertial, trade, services":CommercialTradeServices()})




### Read control parameters

In [16]:
set_and_control_parameters = pd.read_excel(input_path / 'Set_and_Control_Parameters.xlsx')
ctrl = ControlParameters(set_and_control_parameters)
print(ctrl)

sectors_active_values: {'industry': 1, 'households': 1, 'transport': 1, 'commertial, trade, services': 1}
parameter_values: {'Forecast year': 2050, 'Population forecast': 1, 'Timeseries forecast': 0, 'NUTS2 geographical resolution': 0, 'World calculation': 0, 'NUTS2 classification': 2016, 'Graphical output': 0}


### Read available countries

In [17]:
country_names = pd.read_excel(input_path / 'Set_and_Control_Parameters.xlsx', sheet_name="Countries")
Country(country_names)
print(Country)

<class 'countries.Country'>


### Read production data

In [18]:

from collections import namedtuple


gs_param_ind_subs = pd.read_excel(input_path / "Set_and_Control_Parameters.xlsx", sheet_name="IND_subsectors")

product_names = gs_param_ind_subs.get("Main subsectors")

multiply_with_scalar = lambda e, s : e.combine(e.loc[e.index >= 0, e.columns != 'Country'].apply(lambda y : y * s), lambda s1, s2 : s2, overwrite=False)

chem_prod_ren_perc = gs_param_ind_subs.loc[gs_param_ind_subs["Parameter"] == "Percentage of chemical production with renewable H2", "Value"].iloc[0] / 100
dir_steel_perc = gs_param_ind_subs.loc[gs_param_ind_subs["Parameter"] == "Percentage of direct steel in steel production", "Value"].iloc[0] / 100

Retrieve = namedtuple("Retrieve", ["file_name", "sheet_name", "sheet_transform"])
data_access_spec = {"steel": Retrieve("Steel_Production.xlsx", "Data_total", lambda x: x),
                    "steel_prim": Retrieve("Steel_Production.xlsx", "Steel_prim", lambda x: x),
                    "steel_sec": Retrieve("Steel_Production.xlsx", "Steel_sec", lambda x: x),
                    "steel_direct": Retrieve("Steel_Production.xlsx", "Data_total", lambda x: multiply_with_scalar(x, dir_steel_perc)),
                    "alu_prim": Retrieve("Aluminium_Production.xlsx", "Prim_Data", lambda x: x),
                    "alu_sec": Retrieve("Aluminium_Production.xlsx", "Sec_Data_const", lambda x: x),
                    "copper_prim": Retrieve("Copper_Production.xlsx", "Copper_WSP", lambda x : x.loc[x["Type"] == "Primary"].drop("Type", axis=1)),
                    "copper_sec": Retrieve("Copper_Production.xlsx", "Copper_WSP", lambda x : x.loc[x["Type"] == "Secondary"].drop("Type", axis=1)),
                    "chlorine": Retrieve("Chlorin_Production.xlsx", "Data", lambda x: x),
                    "ammonia": Retrieve("Ammonia_Production.xlsx", "Data_const", lambda x : multiply_with_scalar(x, chem_prod_ren_perc)),
                    "methanol": Retrieve("Methanol_Production.xlsx", "Data_const", lambda x : multiply_with_scalar(x, chem_prod_ren_perc)),
                    "ethylene": Retrieve("Ethylene_Production.xlsx", "Data_const", lambda x : multiply_with_scalar(x, chem_prod_ren_perc)),
                    "propylene": Retrieve("Propylene_Production.xlsx", "Data_const", lambda x : multiply_with_scalar(x, chem_prod_ren_perc)),
                    "aromate": Retrieve("Aromate_Production.xlsx", "Data_const", lambda x : multiply_with_scalar(x, chem_prod_ren_perc)),
                    "ammonia_classicammonia_classic": Retrieve("Ammonia_Production.xlsx", "Data_const", lambda x: x),
                    "methanol_classic": Retrieve("Methanol_Production.xlsx", "Data_const", lambda x: x),
                    "ethylene_classic": Retrieve("Ethylene_Production.xlsx", "Data_const", lambda x: x),
                    "propylene_classic": Retrieve("Propylene_Production.xlsx", "Data_const", lambda x: x),
                    "aromate_classic": Retrieve("Aromate_Production.xlsx", "Data_const", lambda x: x),
                    "paper": Retrieve("Paper_Production.xlsx", "Data", lambda x: x),
                    "cement": Retrieve("Cement_Production.xlsx", "Data", lambda x: x),
                    "glass": Retrieve("Glass_Production.xlsx", "Data_const", lambda x: x.drop('Comment', axis=1)),
                    }

SECTOR["industry"].load_production_data(product_names, data_access_spec)


steel                   Country          1967          1968          1969  \
0                  Belgium   9712.000000  11568.000000  12832.000000   
1                  Denmark    401.000000    457.000000    482.000000   
2                   France  19658.000000  20403.000000  22510.000000   
3                  Germany  36744.000000  41159.000000  45316.000000   
4                  Ireland     54.000000     68.000000     81.000000   
5                    Italy  15890.000000  16964.000000  16428.000000   
6               Luxembourg   4461.000000   4834.000000   5521.000000   
7              Netherlands   3401.000000   3706.000000   4712.000000   
8           United Kingdom  24324.000000  26299.000000  26096.000000   
9                  Austria   3023.000000   3467.000000   3926.000000   
10                 Finland    411.000000    729.000000    978.000000   
11                  Greece    210.000000    210.000000    450.000000   
12                  Norway    790.000000    824.000000    8

C:\Users\Chris\Documents\Master_Semester_1\IDP\endemo2\countries.py:42: InvalidCountryName: unidentified countries found in file steel_Production.xlsx
	Unidentified Countries: ['Australia', 'New Zealand']
  warnings.warn("unidentified countries found in file " + file_name + "\n\tUnidentified Countries: " + str(invalid_countries), InvalidCountryName)
C:\Users\Chris\Documents\Master_Semester_1\IDP\endemo2\countries.py:42: InvalidCountryName: unidentified countries found in file steel_prim_Production.xlsx
	Unidentified Countries: ['Australia', 'New Zealand']
  warnings.warn("unidentified countries found in file " + file_name + "\n\tUnidentified Countries: " + str(invalid_countries), InvalidCountryName)


{'steel': {'': <product.Product object at 0x000001CF78168280>, 'prim': <product.Product object at 0x000001CF7821FA00>}}
steel_sec                   Country  1967   1968   1969   1970   1971   1972   1973  \
0                  Belgium   NaN    NaN    NaN    NaN    NaN    NaN    NaN   
1                  Denmark   NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2                   France   NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3                  Germany   NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4                  Ireland   NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5                    Italy   NaN    NaN    NaN    NaN    NaN    NaN    NaN   
6               Luxembourg   NaN    NaN    NaN    NaN    NaN    NaN    NaN   
7              Netherlands   NaN    NaN    NaN    NaN    NaN    NaN    NaN   
8           United Kingdom   NaN    NaN    NaN    NaN    NaN    NaN    NaN   
9                  Austria   NaN    NaN    NaN    NaN    NaN    NaN    NaN   
10           

C:\Users\Chris\Documents\Master_Semester_1\IDP\endemo2\countries.py:42: InvalidCountryName: unidentified countries found in file steel_sec_Production.xlsx
	Unidentified Countries: ['Australia', 'New Zealand']
  warnings.warn("unidentified countries found in file " + file_name + "\n\tUnidentified Countries: " + str(invalid_countries), InvalidCountryName)
C:\Users\Chris\Documents\Master_Semester_1\IDP\endemo2\countries.py:42: InvalidCountryName: unidentified countries found in file steel_direct_Production.xlsx
	Unidentified Countries: ['Australia', 'New Zealand']
  warnings.warn("unidentified countries found in file " + file_name + "\n\tUnidentified Countries: " + str(invalid_countries), InvalidCountryName)


alu_prim                     Country         1986         1987         1988  \
0                    Austria    93.000000    93.000000    95.000000   
1                     France   322.000000   323.000000   328.000000   
2                    Germany   826.000000   861.000000   814.000000   
3                     Greece   124.000000   127.000000   151.000000   
4                    Hungary    74.000000    74.000000    75.000000   
5                    Iceland    76.000000    85.000000    82.000000   
6                      Italy   243.000000   233.000000   226.000000   
7                Netherlands   266.000000   276.000000   278.000000   
8                     Norway   726.000000   853.000000   864.000000   
9                     Poland    48.000000    48.000000    48.000000   
10                   Romania   253.000000   260.000000   266.000000   
11                     Spain   355.000000   341.000000   323.000000   
12                    Sweden    78.000000    81.000000    99.000000  

...

ammonia            2018                 Country
0     785.252105                  France
1    1800.000000             Netherlands
2    2583.012917                 Germany
3     304.979000                   Italy
4     313.392984          United Kingdom
5       0.000000                 Ireland
6       0.223068                 Denmark
7      52.975622                  Greece
8       0.000000                Portugal
9     417.771333                   Spain
10    830.000000                 Belgium
11      0.000000               Luxemburg
12      0.000000                 Iceland
13      0.000000                  Norway
14      0.000000                  Sweden
15      0.000000                 Finland
16    400.000000                 Austria
17      0.000000                   Malta
18     50.676300                 Estonia
19      0.000000                  Latvia
20    763.546307               Lithuania
21   2028.428000                  Poland
22    200.000000                 Czechia
23    399

C:\Users\Chris\Documents\Master_Semester_1\IDP\endemo2\countries.py:42: InvalidCountryName: unidentified countries found in file ammonia_Production.xlsx
	Unidentified Countries: ['Luxemburg', 'Malta', 'EU27TOTALS_2020', 'EUROPEAN UNION (28)']
  warnings.warn("unidentified countries found in file " + file_name + "\n\tUnidentified Countries: " + str(invalid_countries), InvalidCountryName)
C:\Users\Chris\Documents\Master_Semester_1\IDP\endemo2\countries.py:42: InvalidCountryName: unidentified countries found in file methanol_Production.xlsx
	Unidentified Countries: ['Luxemburg', 'Malta']
  warnings.warn("unidentified countries found in file " + file_name + "\n\tUnidentified Countries: " + str(invalid_countries), InvalidCountryName)
C:\Users\Chris\Documents\Master_Semester_1\IDP\endemo2\countries.py:42: InvalidCountryName: unidentified countries found in file ethylene_Production.xlsx
	Unidentified Countries: ['Luxemburg', 'Malta']
  warnings.warn("unidentified countries found in file " + f

propylene           2018                 Country
0   2176.817360                  France
1   2023.047875             Netherlands
2   3838.254500                 Germany
3   1050.000000                   Italy
4      0.000000          United Kingdom
5      0.000000                 Ireland
6      0.000000                 Denmark
7      0.000000                  Greece
8    142.487900                Portugal
9   1219.680600                   Spain
10   992.406069                 Belgium
11     0.000000               Luxemburg
12     0.000000                 Iceland
13   175.260333                  Norway
14   267.882100                  Sweden
15   198.000000                 Finland
16   200.000000                 Austria
17     0.000000                   Malta
18     0.000000                 Estonia
19     0.000000                  Latvia
20    49.660763               Lithuania
21   357.436250                  Poland
22   217.600000                 Czechia
23    33.812517                

C:\Users\Chris\Documents\Master_Semester_1\IDP\endemo2\countries.py:42: InvalidCountryName: unidentified countries found in file propylene_Production.xlsx
	Unidentified Countries: ['Luxemburg', 'Malta']
  warnings.warn("unidentified countries found in file " + file_name + "\n\tUnidentified Countries: " + str(invalid_countries), InvalidCountryName)
C:\Users\Chris\Documents\Master_Semester_1\IDP\endemo2\countries.py:42: InvalidCountryName: unidentified countries found in file aromate_Production.xlsx
	Unidentified Countries: ['Luxemburg', 'Malta']
  warnings.warn("unidentified countries found in file " + file_name + "\n\tUnidentified Countries: " + str(invalid_countries), InvalidCountryName)


KeyError: 'ammonia_classic'

In [ ]:
SECTOR["industry"].add_to_hierarchy("alu_prim", SECTOR["industry"].products["alu_prim"])
SECTOR["industry"].add_to_hierarchy("alu_sec", SECTOR["industry"].products["alu_sec"])
SECTOR["industry"].add_to_hierarchy("alu_sec", SECTOR["industry"].products["alu_sec"])
SECTOR["industry"].add_to_hierarchy("ammonia", SECTOR["industry"].products["ammonia"])
SECTOR["industry"].add_to_hierarchy("ammonia_classic", SECTOR["industry"].products["ammonia_classic"])

SECTOR["industry"].add_to_hierarchy("chlorine", SECTOR["industry"].products["chlorine"])

In [ ]:
prod = SECTOR["industry"].products["alu_prim"]

In [ ]:
dir(prod)